In [2]:
using JLD
include("gyncycle.jl");

In [3]:
SIGMA_RHO = 0.1
SIGMA_Y0 = 100
SIGMA_PARMS = 100;
SIGMA_PROPOSAL = 0.1

parms, y0 = loadparms()
samplingnodes = [:sparms, :y0]
data = loadpfizer()
persons = 1:length(data)

proposalvariance = eye(length(SAMPLEPARMS)+length(y0)) * log(1+(SIGMA_PROPOSAL^2));

In [ ]:
mc = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, [AMM(samplingnodes, proposalvariance, adapt=:none)])
    
    push!(mc, mcmc(m, inp, ini, 1000, burnin=0, verbose = false, thin = 1))
end

while size(mc[1],1) < 1_000_000
    save("13pers-adaptive-correctedhill-0.1-100-100-0.1-flatprior.jld", "mc" , [mc[person].value for person in persons])
    for person in persons
        mc[person] = mcmc(mc[person], 1_000, verbose=false)
    end    
end